In [1]:
import numpy as np

In [2]:
import PIL
from PIL import Image

In [3]:
import matplotlib
import math

In [4]:
import os
import random
import copy
from os import listdir
from collections import defaultdict

In [5]:
def load_data(data_dir="./aligned/"):
    """ Load all PNG images stored in your data directory into a list of NumPy
        arrays.

    Args:
    data_dir: The relative directory path to the CompCar image directory.
    Returns:
        images: A dictionary with keys as car types and a list containing images associated with each key.
        cnt: A dictionary that stores the # of images in each car type
    """
    images = defaultdict(list)

    # Get the list of car type directory:
    for e in listdir(data_dir):
        # excluding any non-directory files
        if not os.path.isdir(os.path.join(data_dir, e)):
            continue
        # Get the list of image file names
        all_files = listdir(os.path.join(data_dir, e))

        for file in all_files:
            # Load only image files as PIL images and convert to NumPy arrays
            if '.jpg' in file:
                img = Image.open(os.path.join(data_dir, e, file))
                images[e].append(np.array(img))

    print("Car types: {} \n".format(list(images.keys())))

    cnt = defaultdict(int)
    for e in images.keys():
        print("{}: {} # of images".format(e, len(images[e])))
        cnt[e] = len(images[e])
    return images, cnt


In [6]:
# this function takes in all the img from all set and return a list of sets of mutually exclusive images and its labels.
def k_fold(imgs, k):
    res = []
    for _ in range(k):
        res.append(defaultdict(list))
    for key in imgs.keys():
        for i, entry in enumerate(imgs[key]):
            res[i%k][key].append(entry)
    return res

In [7]:
def PCA(X, n_components):

    """Args:
        X: has shape Mxd where M is the number of images and d is the dimension of each image
        n_components: The number of components you want to project your image onto. 
    
    Returns:
        projected: projected data of shape M x n_components
        mean_image: mean of all images
        top_sqrt_eigen_values: singular values
        top_eigen_vectors: eigenvectors 
    """
    mean_image = np.average(X, axis = 0)

    msd = X - mean_image # M x d

    smart_cov_matrix = np.matmul(msd, msd.T)
    eigen_values, smart_eigen_vectors = np.linalg.eig(smart_cov_matrix)

    idx = eigen_values.argsort()[::-1]   
    eigen_values = eigen_values[idx]
    smart_eigen_vectors = smart_eigen_vectors[:,idx]

    eigen_vectors = (np.matmul(msd.T, smart_eigen_vectors)).T # M x d

    row_norm = np.sum(np.abs(eigen_vectors)**2,axis=-1)**(1./2) # M

    normalized_eigen_vectors = eigen_vectors/(row_norm.reshape(-1, 1)) # M x d

    top_eigen_vectors = normalized_eigen_vectors[:n_components].T
    top_sqrt_eigen_values = np.sqrt(eigen_values[:n_components])

    projected = np.matmul(msd, top_eigen_vectors)/top_sqrt_eigen_values

    return projected, mean_image, top_sqrt_eigen_values, top_eigen_vectors

In [8]:
img, cnt = load_data()
folds = k_fold(img, 10)
test, validation = folds[0], folds[1]
train = defaultdict(list)
for trainLoaderIter in range(2, 10):
    for key in folds[trainLoaderIter].keys():
        train[key].extend(folds[trainLoaderIter][key])
totalTrainingAmount = 0
for key in train.keys():
    totalTrainingAmount += len(train[key])

Car types: ['Pickup', 'Sedan', 'Minivan', 'Convertible'] 

Pickup: 150 # of images
Sedan: 150 # of images
Minivan: 148 # of images
Convertible: 149 # of images


In [9]:
print(totalTrainingAmount)

477


In [12]:
#newTrainingConvertable = train['Convertible']
X = []
y= []

for i in range(len(train['Convertible'])):
    X.append(train['Convertible'][i].reshape((-1)))
    y.append(0)
for i in range(len(train['Minivan'])):
    X.append(train['Minivan'][i].reshape((-1)))
    y.append(1)
X = np.array(X)
y = np.array(y).reshape(1, len(y)).T
#print(X.shape, y.shape)
#from sklearn.decomposition import PCA as skPCA

#pca = skPCA(n_components=)
#X = pca.fit_transform(X)
#print(X.shape)


1.0

In [14]:
projected, mean_image, top_sqrt_eigen_values, top_eigen_vectors = PCA(X, 20)

In [42]:
class Logistic_regression:
    def __init__(self, dimension, lr):
        self.lr = lr
        self.w = np.zeros(dimension)
        
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))
    
    def caculateGradient(self, X, yTrue):
        yPredict = self.predict(X)
        #print(yPredict, len(yPredict))
        return -X * (yTrue.reshape(-1)- yPredict).reshape((-1,1))*(1/len(X))
    
    def updateWeight(self, X, yTrue):
        self.w -= self.lr * self.caculateGradient(X, yTrue).sum(axis=0)
        
    def predict(self, X):
        return self.sigmoid(X@self.w.T)
    
    def logLikelihood(self, X, yTrue):
        predicted = self.predict(X)
        LLs = y * np.log(predicted) + (1-y) * np.log(1-predicted)
        return np.sum(LLs)
    
    def SGD(self, y_true, y_pred, X):
        self.w = self.w + self.lr * (-X * (y_true.reshape(-1) - y_pred).reshape(y_true.shape[0],1)).sum(axis=0)
    
    def fit(self, X, y, epoch=300):
        log = []
        for _ in range(epoch):
            y_pred = self.predict(X)
            self.SGD(y, y_pred, X)
            #self.updateWeight(X, y)
            print(self.w)
            log.append(self.logLikelihood(X, y))
        return log
    

In [43]:
LR = Logistic_regression(projected.shape[1], 0.5)
LR.fit(projected, y, 300)


[ 0.11594376 -0.05967129 -0.58681158 -0.73543798  0.56462689  1.01713824
 -1.13630573 -0.61936602 -0.87436042  1.20118182  0.53133733  0.25755209
  0.11785075 -0.11459309 -0.63761381  0.19515516  0.27434369  0.63029417
 -0.35084306 -0.38807824]
[ 0.24620478 -0.12655753 -1.24657443 -1.56228625  1.19950639  2.16047497
 -2.41331284 -1.31565215 -1.85729467  2.55153436  1.12848089  0.54690334
  0.25016931 -0.24333361 -1.35423211  0.4147775   0.5829415   1.33886241
 -0.7456462  -0.82431507]
[ 0.39142394 -0.19989021 -1.98556152 -2.48818389  1.91107543  3.43906125
 -3.83916472 -2.09494328 -2.95721782  4.06265523  1.79541032  0.86934052
  0.39694462 -0.38681622 -2.15485195  0.6623363   0.92968214  2.13183893
 -1.19046106 -1.31219258]
[ 0.5513203  -0.27699467 -2.80704966 -3.51625567  2.70329426  4.85421369
 -5.41104409 -2.95955924 -4.17666379  5.73783988  2.53151409  1.22342302
  0.5560824  -0.54456396 -3.03912155  0.94168682  1.31780313  3.01129704
 -1.69181327 -1.85212086]
[ 0.72535225 -0.3538

<ipython-input-42-5ed784966cfc>:22: RuntimeWarning: divide by zero encountered in log
  LLs = y * np.log(predicted) + (1-y) * np.log(1-predicted)
<ipython-input-42-5ed784966cfc>:22: RuntimeWarning: invalid value encountered in multiply
  LLs = y * np.log(predicted) + (1-y) * np.log(1-predicted)


[  92.92396823    1.90396889 -328.96854349 -336.70824748  320.30091529
  502.51640453 -478.24174454 -339.26094954 -457.93664377  574.42195051
  239.00748843  152.45609867  -15.68411724  -52.08115385 -298.68680008
  164.97882352  151.7722039   349.22705635 -247.90880339 -183.67370778]
[  93.33992447    1.93160967 -330.42741998 -338.10679972  321.71048218
  504.69250218 -480.27253284 -340.77959573 -459.92420209  576.90113297
  240.03429132  153.1455386   -15.79068338  -52.30733954 -299.96707164
  165.74377941  152.45630489  350.78408067 -248.98745035 -184.47183647]
[  93.75586985    1.95942953 -331.88669625 -339.5050578   323.12021479
  506.86839745 -482.30298739 -342.29863626 -461.91146815  579.38053059
  241.06095839  153.83484406  -15.897474    -52.53374094 -301.24718806
  166.50883961  153.14069783  352.34129717 -250.066004   -185.27015663]
[  94.17180437    1.98743145 -333.34637885 -340.9030216   324.53011278
  509.0440859  -484.33310697 -343.81807449 -463.89843824  581.86014747
  2

[-39160.81719524302,
 -39947.295273732016,
 -41447.22663897622,
 -43785.290784326506,
 -47027.1145780399,
 -51168.07218271535,
 -56143.5026353891,
 -61851.64246396883,
 -68177.39851773396,
 -75009.60392843654,
 -82250.27735185102,
 -89817.74332248972,
 -97646.17148145913,
 -105683.51290029028,
 -113889.0088947349,
 -122230.84298815895,
 -130684.15004822653,
 -139229.4183575995,
 -147851.24533796863,
 -156537.3846516323,
 -165278.02253000916,
 -174065.22985289246,
 -182892.54706592974,
 -191754.6687662517,
 -200647.2029064373,
 -209566.48579524193,
 -218509.4390922939,
 -227473.4584473407,
 -236456.326266924,
 -245456.14183469227,
 -254471.26691049407,
 -263500.28306374606,
 -272541.9524495027,
 -281595.18192420807,
 -290659.06254900416,
 -299732.6929321699,
 -308815.35296781163,
 -317906.63063243945,
 -327004.59995998367,
 -336110.03009172727,
 -345221.9012425745,
 -354332.4144742425,
 -363433.50083026465,
 -372542.3021141356,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,

In [46]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(projected, y.reshape((-1)))
reg.score(projected, y)

0.8523206751054853

In [47]:


#(a) Implement Logistic Regression via Gradient Descent. (5 points)
#Now, without using any high-level machine learning libraries, implement logistic regression. Here, you’ll be using batch gradient descent, and will only need one logistic output unit. (Think about why we only need one if we’re classifying two classes?) The most points will be given for clean, well-documented code.
def gradient_descent(X, y, lr=0.05, epoch=300):
    '''
    Gradient Descent for a single feature
    X -> feature -> m * d
    y -> label -> m
    lr -> learning rate
    epoch -> epoch
    
    weights <-
    mse     <-
    '''
    
    weight = np.zeros((X.shape[1], 1))
    N = len(X) # number of samples
    log = []
    
    for _ in range(epoch):
        errorSum = np.zeros((X.shape[1], 1))
        #biasIncludedX = np.append(X, np.ones((X.shape[0], 1)), 1)
        calculatedTarget = X@weight
        #print(calculatedTarget, y)
        diffs = y - calculatedTarget
        #print(diffs.shape, y.shape, calculatedTarget.shape)
        for i, diff in enumerate(diffs):
            #print(diff.shape, errorSum.shape, X[i].shape)
            #print(diff[0])
            errorSum += diff[0] * X[i].reshape((-1, 1))
        weight += errorSum * lr * (2/N)
        #print(weight)
        log.append(sum(diffs**2)[0])

    return weight, log

In [44]:
w, l = gradient_descent(X, y)

<ipython-input-41-049695596806>:32: RuntimeWarning: overflow encountered in square
  log.append(sum(diffs**2)[0])
<ipython-input-41-049695596806>:29: RuntimeWarning: overflow encountered in multiply
  errorSum += diff[0] * X[i].reshape((-1, 1))
<ipython-input-41-049695596806>:29: RuntimeWarning: overflow encountered in add
  errorSum += diff[0] * X[i].reshape((-1, 1))
<ipython-input-41-049695596806>:22: RuntimeWarning: invalid value encountered in matmul
  calculatedTarget = X@weight


In [45]:
print(l)

[118.0, 3.104076254465395e+17, 1.673244493858922e+33, 9.020297129799019e+48, 4.8627539270974253e+64, 2.6214630644143286e+80, 1.4132050893618976e+96, 7.618450367312572e+111, 4.107032053317335e+127, 2.2140607963199377e+143, 1.1935785126978462e+159, 6.434464981006501e+174, 3.4687571157943585e+190, 1.8699730224488486e+206, 1.0080841603940608e+222, 5.434483076694745e+237, 2.929676655105549e+253, 1.5793600205101066e+269, 8.514175344363435e+284, 4.589908624580285e+300, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [94]:
def PCA_test(X, mean_image, n_components):
    msd = X - mean_image # M x d

    smart_cov_matrix = np.matmul(msd, msd.T)
    eigen_values, smart_eigen_vectors = np.linalg.eig(smart_cov_matrix)

    idx = eigen_values.argsort()[::-1]   
    eigen_values = eigen_values[idx]
    smart_eigen_vectors = smart_eigen_vectors[:,idx]

    eigen_vectors = (np.matmul(msd.T, smart_eigen_vectors)).T # M x d

    row_norm = np.sum(np.abs(eigen_vectors)**2,axis=-1)**(1./2) # M

    normalized_eigen_vectors = eigen_vectors/(row_norm.reshape(-1, 1)) # M x d

    top_eigen_vectors = normalized_eigen_vectors[:n_components].T
    top_sqrt_eigen_values = np.sqrt(eigen_values[:n_components])

    projected = np.matmul(msd, top_eigen_vectors)/top_sqrt_eigen_values

    return projected

In [146]:
pca = PCA(n_components=4)
pca.fit()
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.9513201441478435

In [147]:
print(reg.predict(X))

[[ 1.09062380e+00]
 [ 1.13370466e+00]
 [ 9.96141150e-01]
 [ 9.79065039e-01]
 [ 8.32071519e-01]
 [ 1.00414370e+00]
 [ 1.03011653e+00]
 [ 9.09847423e-01]
 [ 9.47712967e-01]
 [ 8.89817480e-01]
 [ 1.01802904e+00]
 [ 6.35254183e-01]
 [ 1.02264444e+00]
 [ 9.29018337e-01]
 [ 1.14174318e+00]
 [ 8.92076144e-01]
 [ 1.00485116e+00]
 [ 9.72286533e-01]
 [ 9.42922905e-01]
 [ 1.02190039e+00]
 [ 9.62478241e-01]
 [ 9.68452507e-01]
 [ 9.25448859e-01]
 [ 8.42111768e-01]
 [ 8.65834077e-01]
 [ 9.91952875e-01]
 [ 8.93742155e-01]
 [ 8.17413421e-01]
 [ 9.86742253e-01]
 [ 1.04941998e+00]
 [ 8.07144457e-01]
 [ 8.91723287e-01]
 [ 9.82676583e-01]
 [ 1.06600742e+00]
 [ 9.11307601e-01]
 [ 1.12803576e+00]
 [ 9.58403864e-01]
 [ 8.86231427e-01]
 [ 9.75306789e-01]
 [ 1.04724480e+00]
 [ 9.31669463e-01]
 [ 1.04328470e+00]
 [ 1.22560873e+00]
 [ 1.08971834e+00]
 [ 9.41061064e-01]
 [ 1.03373186e+00]
 [ 8.83190078e-01]
 [ 9.75095138e-01]
 [ 1.05218043e+00]
 [ 1.10345402e+00]
 [ 1.04831977e+00]
 [ 9.53203755e-01]
 [ 9.8189797

In [141]:

#(a) Implement Logistic Regression via Gradient Descent. (5 points)
#Now, without using any high-level machine learning libraries, implement logistic regression. Here, you’ll be using batch gradient descent, and will only need one logistic output unit. (Think about why we only need one if we’re classifying two classes?) The most points will be given for clean, well-documented code.
def gradient_descent(X, y, lr=0.05, epoch=300):
    '''
    Gradient Descent for a single feature
    X -> feature -> m * d
    y -> label -> m
    lr -> learning rate
    epoch -> epoch
    
    weights <-
    mse     <-
    '''
    
    weight = np.zeros((projected.shape[1], 1))
    N = len(X) # number of samples
    log = []
    #log, mse = [], [] # lists to store learning process
    # M * (d+1)--> X with last be bias term (feature)
    # M * 1    --> y --> lable
    
    for _ in range(epoch):
        errorSum = np.zeros((X.shape[1], 1))
        #biasIncludedX = np.append(X, np.ones((X.shape[0], 1)), 1)
        calculatedTarget = X@weight
        #print(calculatedTarget, y)
        diffs = y - calculatedTarget
        #print(diffs.shape, y.shape, calculatedTarget.shape)
        for i, diff in enumerate(diffs):
            #print(diff.shape, errorSum.shape, X[i].shape)
            #print(diff[0])
            errorSum += diff[0] * X[i].reshape((-1, 1))
        weight += errorSum * lr * (2/N)
        log.append(sum(diffs**2)[0])

    return weight, log

In [142]:
#newTrainingMiniVan = train['Minivan']
#newTrainingConvertable = train['Convertible']
y = []
X = []
for i in range(len(memo)):
    if memo[i][0] == 'Convertible':
        y.append(0)
        X.append(projected[i])
    elif memo[i][0] == 'Minivan':
        y.append(1)
        X.append(projected[i])
X = np.array(X)
y = np.array(y).reshape(1, len(y)).T

In [143]:
print(X.shape)
print(y.shape)

(237, 200)
(237, 1)


In [145]:
w, l = gradient_descent(X, y)
print(l)

[118.0, 117.95204991025547, 117.9041295251166, 117.85623882400905, 117.80837778637338, 117.76054639166554, 117.712744619356, 117.66497244893043, 117.61722985988915, 117.56951683174813, 117.52183334403742, 117.47417937630263, 117.42655490810381, 117.37895991901632, 117.33139438862993, 117.2838582965497, 117.23635162239532, 117.18887434580154, 117.14142644641748, 117.09400790390765, 117.04661869795116, 116.9992588082418, 116.95192821448835, 116.90462689641403, 116.8573548337573, 116.81011200627113, 116.7628983937231, 116.71571397589584, 116.66855873258662, 116.62143264360728, 116.57433568878429, 116.5272678479591, 116.48022910098766, 116.43321942774074, 116.3862388081036, 116.33928722197616, 116.2923646492733, 116.24547106992397, 116.19860646387197, 116.151770811076, 116.10496409150906, 116.0581862851587, 116.01143737202716, 115.96471733213139, 115.91802614550262, 115.87136379218653, 115.82473025224371, 115.77812550574913, 115.73154953279189, 115.68500231347629, 115.63848382792054, 115.5

In [58]:
validationConvertable = test['Convertible']
validationMinivan     = test['Minivan']

validationPCAed = []
validationY = []
for i in range(len(validationConvertable)):
    validationPCAed.append((np.matmul(validationConvertable[i].reshape(1, 60000) - mean_image, top_eigen_vectors)/top_sqrt_eigen_values)[0])
    validationY.append(0)
for i in range(len(validationMinivan)):
    validationPCAed.append((np.matmul(validationMinivan[i].reshape(1, 60000) - mean_image, top_eigen_vectors)/top_sqrt_eigen_values)[0])
    validationY.append(1)

In [ ]:
print(len(validationPCAed))
validationPCAed = np.array(validationPCAed)
print(validationPCAed.shape[0])

In [ ]:
validationPCAed = np.append(validationPCAed, np.ones((validationPCAed.shape[0], 1)), 1)

In [ ]:
validationPCAed@w

In [ ]:
validationPCAed